In [3]:
import os
os.environ["GOOGLE_API_KEY"] = "XXXXX"
# The key you want to access must match the name you set in your OS
api_key = os.environ.get("GOOGLE_API_KEY")

if api_key:
    print(f"API Key successfully loaded: {api_key}")
    # Now you can use this 'api_key' variable in your Google API requests
else:
    print("API Key not found in environment variables.")

API Key successfully loaded: XXXXX


In [ ]:
# Blogger API Setup
# You'll need to set up OAuth2 credentials for Blogger API
# Follow: https://developers.google.com/blogger/docs/3.0/using

# Set your Blogger blog ID here
BLOGGER_BLOG_ID = "xxxx"  # Replace with your actual blog ID

# For OAuth2 authentication, you'll need to download credentials from Google Cloud Console
# and save them as 'blogger_credentials.json'
BLOGGER_CREDENTIALS_FILE = "blogger_credentials.json"

In [3]:
import uuid
from google.genai import types

    
from google.adk.agents import Agent,SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner,InMemoryRunner
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService

from google.adk.tools import google_search,AgentTool
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

# Additional imports for Blogger and PDF processing
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pickle
import PyPDF2
import re
import time
from PIL import Image

import openai
from openai import OpenAI
import base64

In [2]:
OPENAI_API_KEY = "XXX"  # 🔥 REPLACE WITH YOUR ACTUAL KEY
client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ OpenAI client initialized")
print(OPENAI_API_KEY)

NameError: name 'OpenAI' is not defined

In [6]:
#When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. 
#Retry options automatically handle these failures by retrying the request with exponential backoff.
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [7]:
Blog_content = Agent(
    name="Blog_content",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that uses google search to get blog content for the given topic",
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. 3 main sections
    3. A concluding thought""",
    tools=[google_search],
    output_key="Blog_content_output",
)

In [8]:
Blog_image_finder = Agent(
    name="blog_image_finder",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Finds relevant images for each section of the blog outline using Google Search",
    instruction="""You will receive a blog outline with 3 main sections. Your task is to:
1. For each section, use Google Search to find relevant image based on section name (search for "[section name] images" or "stock photos [section name]")
2. Look for images in Dall e only
3. Provide ONE specific image recommendation for EACH of the 3 sections with source information

Format your output as:
Section 1: [section name]
- Recommended image: [description in 15 words]
- Source suggestion: [website/source]

Section 2: [section name]
- Recommended image: [description in 15 words]
- Source suggestion: [website/source]

Section 3: [section name]
- Recommended image: [description in 15 words]
- Source suggestion: [website/source]""",
    tools=[google_search],  # Use regular google_search
    output_key="Blog_images_output",
)

print("✅ Blog_image_finder agent created.")

✅ Blog_image_finder agent created.


In [9]:
Blog_writer = Agent(
    name="Blog_writer",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Writes a complete, engaging blog post using the outline and image suggestions",
    instruction="""You will receive:
1. A blog outline with headline, 3 main sections, and conclusion
2. Image recommendations for each section

Your task is to write a complete, engaging blog post that:
1. Uses the provided headline or creates an improved version
2. Writes an engaging introduction (2-3 paragraphs) that hooks the reader
3. For each of the 3 main sections:
   - Expand the section into 3-4 detailed paragraphs
   - Include the recommended image placement with [IMAGE: description - Source: website]
   - Use subheadings, examples, and clear explanations
   - Make it informative and engaging
4. Write a strong conclusion (2-3 paragraphs) that:
   - Summarizes key points
   - Provides actionable takeaways
   - Ends with a call-to-action or thought-provoking question

Writing style:
- Professional but conversational tone
- Use short paragraphs (3-4 sentences max)
- Include transitions between sections
- Target 1200-1500 words total
- SEO-friendly with natural keyword usage

Format the output as a complete blog post ready for publication.""",
    tools=[],  # No tools needed - just writing
    output_key="final_blog_post",
)

print("✅ Blog_writer agent created.")

✅ Blog_writer agent created.


In [10]:
Blog_PDF_formatter = Agent(
    name="Blog_PDF_formatter",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Formats the blog post content for PDF generation with clear structure",
    instruction="""You will receive a complete blog post.

Your task is to extract and structure the content for PDF generation:

1. Extract the main title/headline
2. Identify all section headings (usually 3 main sections)
3. Separate all paragraphs
4. Identify image placeholders and their descriptions


Format your output EXACTLY as follows:

TITLE: [Extract the main blog title/headline]

SECTION: [First section heading]
CONTENT: [All paragraphs for this section, separated by ||]
IMAGE: [Image description]

SECTION: [Second section heading]
CONTENT: [All paragraphs for this section, separated by ||]
IMAGE: [Image description]

SECTION: [Third section heading]
CONTENT: [All paragraphs for this section, separated by ||]
IMAGE: [Image description]

CONCLUSION: [Conclusion heading if any]
CONTENT: [Conclusion paragraphs separated by ||]

IMPORTANT: 
- Use || to separate different paragraphs within CONTENT
- Keep the exact format with TITLE:, SECTION:, CONTENT:, IMAGE: labels
- Do not add any extra formatting or markdown
- Extract content as plain text""",
    tools=[],
    output_key="pdf_formatted_content",
)

print("✅ Blog_PDF_formatter agent created.")

✅ Blog_PDF_formatter agent created.


In [11]:
content_agent = SequentialAgent(
    name="Blogwriter",
    sub_agents=[Blog_content,Blog_image_finder],
)

print("✅ content_agent created.")

✅ content_agent created.


In [12]:
root_agent = SequentialAgent(
    name="Blogwriter",
    sub_agents=[content_agent,Blog_writer],
)

In [13]:
root_pdf = SequentialAgent(
    name="Blogwriter",
    sub_agents=[root_agent,Blog_PDF_formatter],
)

In [14]:
# ============================================
# Cell: Import Required Libraries
# ============================================
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER
from reportlab.lib import colors
from datetime import datetime
from io import BytesIO
from PIL import Image as PILImage, ImageDraw, ImageFont
import requests
import time
import os

print("✅ Libraries imported")

✅ Libraries imported


In [ ]:
# ============================================
# Cell: Set Pexels API Key
# ============================================
# Get your FREE key at: https://unsplash.com/developers
PEXELS_ACCESS_KEY = "XXXX"

print("✅ API key configured")


✅ API key configured


In [16]:
import time
import requests
from io import BytesIO

def fetch_dalle_image_tool(query: str, openai_key: str = None, width: int = 1024, height: int = 1024, style: str = "natural") -> dict:
    """
    Generate images using DALL-E 3 with improved prompts for better quality
    
    Args:
        query: Image description/prompt for DALL-E
        openai_key: OpenAI API key (uses global if not provided)
        width: Image width (default: 1024)
        height: Image height (default: 1024)
        style: "vivid" for hyper-realistic or "natural" for more subdued (default: "vivid")
    
    Returns: dict with success status and image data
    """
    try:
        # Clean up the query
        clean_query = query.replace('IMAGE:', '').strip()
        
        # Extract description before "Source:" if present
        if ' - Source:' in clean_query:
            clean_query = clean_query.split(' - Source:')[0].strip()
        elif '- Source:' in clean_query:
            clean_query = clean_query.split('- Source:')[0].strip()
        
        # IMPROVED PROMPT ENHANCEMENT
        # Different enhancement based on content type
        if any(word in clean_query.lower() for word in ['fashion', 'clothing', 'style', 'outfit', 'wear']):
            # Fashion-focused prompt
            enhanced_prompt = f"{clean_query}. Professional fashion photography, high-end editorial style, shot with Canon EOS R5, 85mm f/1.4 lens, shallow depth of field, natural lighting, vibrant colors, ultra sharp details, 8K resolution, cinematic composition"
        
        elif any(word in clean_query.lower() for word in ['street', 'urban', 'city', 'graffiti']):
            # Urban/street photography prompt
            enhanced_prompt = f"{clean_query}. Professional street photography, dynamic composition, vibrant urban atmosphere, golden hour lighting, rich colors, authentic street culture, high detail, shot with Sony A7R IV, 35mm lens, photojournalistic style, 8K quality"
        
        elif any(word in clean_query.lower() for word in ['office', 'workspace', 'professional', 'business']):
            # Professional/corporate prompt
            enhanced_prompt = f"{clean_query}. Professional corporate photography, modern aesthetic, clean composition, natural window lighting, architectural photography style, high-end interior design, ultra sharp, 8K resolution, shot with Nikon Z9"
        
        elif any(word in clean_query.lower() for word in ['food', 'restaurant', 'cafe', 'coffee', 'dining']):
            # Food/restaurant prompt
            enhanced_prompt = f"{clean_query}. Professional food photography, appetizing presentation, warm ambient lighting, shallow depth of field, rustic yet modern aesthetic, shot with Canon EOS R6, 100mm macro lens, mouthwatering details, 8K quality"
        
        elif any(word in clean_query.lower() for word in ['product', 'gadget', 'device', 'tech']):
            # Product photography prompt
            enhanced_prompt = f"{clean_query}. Professional product photography, studio lighting, clean white background with subtle shadows, ultra sharp details, commercial quality, shot with Phase One XF IQ4, perfect focus, 8K resolution, advertising style"
        
        else:
            # General high-quality prompt
            enhanced_prompt = f"{clean_query}. Professional photography, masterful composition, perfect lighting, vibrant yet natural colors, ultra sharp details, photorealistic, 8K resolution, award-winning quality, shot with professional camera equipment"
        
        # Add Indian context if relevant and not already mentioned
        if 'india' in clean_query.lower() or 'indian' in clean_query.lower():
            pass  # Already mentioned
        elif any(word in clean_query.lower() for word in ['street', 'fashion', 'style', 'people', 'urban']):
            enhanced_prompt += ", Indian context and setting"
        
        # Truncate if too long
        if len(enhanced_prompt) > 3900:
            enhanced_prompt = enhanced_prompt[:3900] + "..."
        
        print(f"🎨 Generating DALL-E 3 image...")
        print(f"📝 Description: {clean_query[:80]}...")
        
        # Use the OpenAI client
        if openai_key:
            import openai
            local_client = openai.OpenAI(api_key=openai_key)
        else:
            try:
                local_client = client
            except NameError:
                try:
                    import openai
                    local_client = openai.OpenAI(api_key=OPENAI_API_KEY)
                except:
                    return {
                        "success": False,
                        "error": "OpenAI client not initialized. Please run Cell 4 first."
                    }
        
        # Determine size
        if width == 1024 and height == 1792:
            size = "1024x1792"  # Portrait
        elif width == 1792 and height == 1024:
            size = "1792x1024"  # Landscape
        else:
            size = "1024x1024"  # Square (default)
        
        # Generate image with DALL-E 3
        response = local_client.images.generate(
            model="dall-e-3",
            prompt=enhanced_prompt,
            size=size,
            quality="hd",  # Always use HD for best quality
            style=style,   # "vivid" for hyper-realistic or "natural"
            n=1
        )
        
        # Get image URL and download
        image_url = response.data[0].url
        print(f"⬇️  Downloading high-quality image...")
        img_resp = requests.get(image_url, timeout=30)
        
        if img_resp.status_code == 200:
            print(f"✅ High-quality DALL-E 3 image generated!")
            return {
                "success": True,
                "image_data": img_resp.content,
                "url": image_url,
                "prompt": enhanced_prompt,
                "revised_prompt": response.data[0].revised_prompt,
                "generator": "DALL-E 3",
                "style": style,
                "quality": "HD"
            }
        else:
            return {"success": False, "error": "Failed to download generated image"}
    
    except Exception as e:
        error_msg = str(e)
        print(f"❌ DALL-E error: {error_msg}")
        
        if "insufficient_quota" in error_msg.lower():
            return {"success": False, "error": "OpenAI quota exceeded. Add credits at platform.openai.com/account/billing"}
        elif "invalid_api_key" in error_msg.lower() or "incorrect api key" in error_msg.lower():
            return {"success": False, "error": "Invalid OpenAI API key. Check Cell 4."}
        elif "rate_limit" in error_msg.lower():
            return {"success": False, "error": "Rate limit exceeded. Wait 60 seconds."}
        else:
            return {"success": False, "error": error_msg}


# Backward compatibility alias
def fetch_pexels_image_tool(query: str, pexels_key: str = None, width: int = 1024, height: int = 1024) -> dict:
    """Now uses DALL-E instead of Pexels"""
    return fetch_dalle_image_tool(query, openai_key=None, width=width, height=height, style="vivid")


print("✅ IMPROVED fetch_dalle_image_tool function defined")
print("✅ Enhanced with professional photography prompts")
print("✅ fetch_pexels_image_tool alias created")


✅ IMPROVED fetch_dalle_image_tool function defined
✅ Enhanced with professional photography prompts
✅ fetch_pexels_image_tool alias created


In [17]:
def create_styled_placeholder_tool(text: str, width: int = 600, height: int = 400) -> dict:
    """
    Generate a professional placeholder with gradient background
    
    Args:
        text: Text to display on placeholder
        width: Image width in pixels
        height: Image height in pixels
    
    Returns: dict with success status and image data
    """
    try:
        img = PILImage.new('RGB', (width, height))
        pixels = img.load()
        
        # Create blue gradient
        for y in range(height):
            for x in range(width):
                progress = y / height
                r = int(70 + progress * 100)
                g = int(110 + progress * 80)
                b = int(180 + progress * 60)
                pixels[x, y] = (r, g, b)
        
        draw = ImageDraw.Draw(img)
        
        # Try to load better fonts, fallback to default
        try:
            font_large = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", 24)
            font_small = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", 16)
        except:
            try:
                font_large = ImageFont.truetype("arial.ttf", 24)
                font_small = ImageFont.truetype("arial.ttf", 16)
            except:
                font_large = ImageFont.load_default()
                font_small = ImageFont.load_default()
        
        # Add emoji icon at top
        icon = "🖼️"
        icon_bbox = draw.textbbox((0, 0), icon, font=font_large)
        icon_w = icon_bbox[2] - icon_bbox[0]
        draw.text(((width - icon_w) // 2, 80), icon, fill="white", font=font_large)
        
        # Wrap text into multiple lines
        words = text[:80].split()
        lines = []
        current_line = []
        
        for word in words:
            test_line = ' '.join(current_line + [word])
            bbox = draw.textbbox((0, 0), test_line, font=font_small)
            line_width = bbox[2] - bbox[0]
            
            if line_width < width - 100:
                current_line.append(word)
            else:
                if current_line:
                    lines.append(' '.join(current_line))
                current_line = [word]
        
        if current_line:
            lines.append(' '.join(current_line))
        
        # Draw text centered with shadow
        y_position = 140
        for line in lines[:4]:
            bbox = draw.textbbox((0, 0), line, font=font_small)
            line_width = bbox[2] - bbox[0]
            x_position = (width - line_width) // 2
            
            draw.text((x_position + 1, y_position + 1), line, fill=(0, 0, 0, 128), font=font_small)
            draw.text((x_position, y_position), line, fill="white", font=font_small)
            y_position += 28
        
        # Add footer
        footer = "Image placeholder"
        footer_bbox = draw.textbbox((0, 0), footer, font=font_small)
        footer_w = footer_bbox[2] - footer_bbox[0]
        draw.text(((width - footer_w) // 2, height - 40), footer, fill=(200, 200, 200), font=font_small)
        
        # Save to buffer
        buffer = BytesIO()
        img.save(buffer, format="PNG")
        buffer.seek(0)
        
        print(f"✅ Styled placeholder created")
        return {
            "success": True,
            "image_data": buffer.getvalue()
        }
        
    except Exception as e:
        print(f"⚠️ Placeholder creation error: {e}")
        return {"success": False, "error": str(e)}


In [18]:
def parse_formatted_content_tool(formatted_content: str) -> dict:
    """
    Parse structured content from Blog_PDF_formatter agent
    
    Expected format:
        TITLE: [title]
        SECTION: [section name]
        CONTENT: [para1||para2||para3]
        IMAGE: [description - Source: source]
    
    Returns: dict with 'title' and 'sections' list
    """
    data = {
        'title': '',
        'sections': []
    }
    
    lines = formatted_content.strip().split('\n')
    current_section = None
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        if line.startswith('TITLE:'):
            data['title'] = line.replace('TITLE:', '').strip()
            
        elif line.startswith('SECTION:'):
            if current_section:
                data['sections'].append(current_section)
            current_section = {
                'heading': line.replace('SECTION:', '').strip(),
                'content': [],
                'images': []
            }
            
        elif line.startswith('CONTENT:'):
            if current_section:
                content = line.replace('CONTENT:', '').strip()
                paragraphs = content.split('||')
                current_section['content'] = [p.strip() for p in paragraphs if p.strip()]
                
        elif line.startswith('IMAGE:'):
            if current_section:
                current_section['images'].append(line.replace('IMAGE:', '').strip())
                
        elif line.startswith('CONCLUSION:'):
            if current_section:
                data['sections'].append(current_section)
            current_section = {
                'heading': line.replace('CONCLUSION:', '').strip() or 'Conclusion',
                'content': [],
                'images': []
            }
    
    if current_section:
        data['sections'].append(current_section)
    
    return data

In [19]:
def generate_pdf_tool(
    formatted_content: str,
    filename: str = "blog_post.pdf",
    pexels_key: str = "BFQ6UOOEVrvMyO7fIGSycDf7SklNwypkUYmcn4LwLGX1NkLQ4jtF1GLZ"  # Change parameter name
) -> dict:
    """
    Generate professional PDF with embedded images from Pexels
    
    Args:
        formatted_content: Structured content from Blog_PDF_formatter agent
        filename: Output PDF filename
        pexels_key: Pexels API access key
    
    Returns: dict with success status and filename
    """
    print("**********input received*********")
    print(formatted_content)
    print("***********Input ends*************")
    try:
        if not pexels_key:
            print("⚠️ Warning: No Pexels key provided. Will use placeholders only.")
        
        # Parse the content
        data = parse_formatted_content_tool(formatted_content)
        
        # Create PDF document
        doc = SimpleDocTemplate(
            filename,
            pagesize=letter,
            rightMargin=72,
            leftMargin=72,
            topMargin=72,
            bottomMargin=50
        )
        
        story = []
        styles = getSampleStyleSheet()
        
        # Define custom styles (same as before)
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=28,
            textColor=colors.HexColor('#2C3E50'),
            spaceAfter=30,
            spaceBefore=20,
            alignment=TA_CENTER,
            fontName='Helvetica-Bold',
            leading=34
        )
        
        heading_style = ParagraphStyle(
            'CustomHeading',
            parent=styles['Heading2'],
            fontSize=18,
            textColor=colors.HexColor('#34495E'),
            spaceAfter=15,
            spaceBefore=25,
            fontName='Helvetica-Bold'
        )
        
        body_style = ParagraphStyle(
            'CustomBody',
            parent=styles['BodyText'],
            fontSize=11,
            leading=16,
            alignment=TA_JUSTIFY,
            spaceAfter=12,
            fontName='Helvetica'
        )
        
        image_caption_style = ParagraphStyle(
            'ImageCaption',
            parent=styles['Italic'],
            fontSize=9,
            textColor=colors.HexColor('#7F8C8D'),
            spaceAfter=20,
            spaceBefore=5,
            alignment=TA_CENTER,
            fontName='Helvetica-Oblique'
        )
        
        date_style = ParagraphStyle(
            'DateStyle',
            parent=styles['Normal'],
            fontSize=10,
            textColor=colors.HexColor('#95A5A6'),
            alignment=TA_CENTER,
            spaceAfter=40,
            fontName='Helvetica-Oblique'
        )
        
        # Build PDF content
        date_text = f"Generated on {datetime.now().strftime('%B %d, %Y')}"
        story.append(Paragraph(date_text, date_style))
        
        if data['title']:
            story.append(Paragraph(data['title'], title_style))
            story.append(Spacer(1, 0.3 * inch))
        
        # Add sections
        for section in data['sections']:
            if section['heading']:
                story.append(Paragraph(section['heading'], heading_style))
            
            for para in section['content']:
                if para:
                    story.append(Paragraph(para, body_style))
            
            # Section images
            for image_info in section['images']:
                parts = image_info.split('- Source:')
                image_desc = parts[0].strip()
                source = parts[1].strip() if len(parts) > 1 else 'Pexels'  # Changed default
                
                # Extract search query
                search_query = image_desc.split('-')[0].strip()
                search_query = search_query.replace('IMAGE:', '').strip()
                key_terms = ' '.join(search_query.split()[:15])
                
                print(f"\n🖼️ Processing: {key_terms[:150]}...")
                
                # Try to fetch image from Pexels
                image_buffer = None
                if pexels_key:
                    result = fetch_pexels_image_tool(key_terms, pexels_key)  # Changed function
                    if result['success']:
                        image_buffer = BytesIO(result['image_data'])
                
                # Fallback to placeholder
                if not image_buffer:
                    print("⚠️ Creating placeholder...")
                    placeholder_result = create_styled_placeholder_tool(search_query)
                    if placeholder_result['success']:
                        image_buffer = BytesIO(placeholder_result['image_data'])
                
                if image_buffer:
                    try:
                        img = RLImage(image_buffer, width=5*inch, height=3.33*inch)
                        story.append(Spacer(1, 0.15 * inch))
                        story.append(img)
                        
                        caption = f"{image_desc} | Source: {source}"
                        story.append(Paragraph(caption, image_caption_style))
                        
                    except Exception as e:
                        print(f"❌ Error embedding image: {e}")
                        fallback_text = f"📷 {image_info}"
                        story.append(Paragraph(fallback_text, image_caption_style))
                else:
                    fallback_text = f"📷 {image_info}"
                    story.append(Paragraph(fallback_text, image_caption_style))
            
            story.append(Spacer(1, 0.25 * inch))
        
        # Build and save PDF
        doc.build(story)
        print(f"\n✅ PDF successfully generated: {filename}")
        
        return {
            "success": True,
            "filename": filename,
            "path": os.path.abspath(filename)
        }
        
    except Exception as e:
        print(f"\n❌ Error generating PDF: {e}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": str(e)
        }


In [20]:
# ============================================
# STEP 2: Create Function Tools
# ============================================

fetch_image_tool = FunctionTool(fetch_pexels_image_tool)
create_placeholder_tool = FunctionTool(create_styled_placeholder_tool)
parse_content_tool = FunctionTool(parse_formatted_content_tool)
generate_pdf_function_tool = FunctionTool(generate_pdf_tool)

In [21]:
# Image Fetcher Agent
Image_Fetcher_Agent = Agent(
    name="Image_Fetcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Fetches images from Unsplash or creates placeholders",
    instruction="""You are an image fetching specialist. Your role is to:
    1. Extract search queries from image descriptions
    2. Use the fetch_unsplash_image tool to get real images from Unsplash
    3. If Unsplash fails, use create_placeholder tool to generate styled placeholders
    4. Return image data and metadata
    
    Always try Unsplash first, then fall back to placeholders if needed.""",
    tools=[fetch_image_tool, create_placeholder_tool],
    output_key="fetched_images"
)

# Content Parser Agent
Content_Parser_Agent = Agent(
    name="Content_Parser",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Parses formatted blog content into structured data",
    instruction="""You are a content parsing specialist. Your role is to:
    1. Receive formatted blog content from the PDF formatter agent
    2. Use the parse_content tool to extract title, sections, paragraphs, and image placeholders
    3. Structure the data properly for PDF generation
    4. Return parsed content in a clean format
    
    Ensure all sections, content, and images are properly extracted.""",
    tools=[parse_content_tool],
    output_key="parsed_content"
)

# PDF Generator Agent
PDF_Generator_Agent = Agent(
    name="PDF_Generator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Generates professional PDF documents with embedded images from Pexels",  # Updated
    instruction="""You are a PDF generation specialist. Your role is to:
    1. Receive parsed blog content and Pexels API key  # Changed
    2. Use the generate_pdf tool to create a professional PDF document
    3. Fetch images using the Pexels API for each section  # Changed
    4. Apply professional styling with custom fonts and colors
    5. Return the PDF filename and path
    
    Requirements:
    - Use 5x3.33 inch images
    - Apply gradient placeholders if images fail
    - Include image captions with source attribution (Pexels)  # Changed
    - Maintain professional typography and spacing
    - Generate a date stamp on the first page
    
    Input format expected:
    - formatted_content: The structured blog content
    - filename: Output PDF filename (default: "blog_post.pdf")
    - pexels_key: Pexels API access key  # Changed
    
    Output format:
    - success: boolean
    - filename: string
    - path: absolute file path
    - error: string (if failed)""",
    tools=[generate_pdf_function_tool],
    output_key="pdf_result"
)

print("✅ All agents updated to use Pexels API!")

print("✅ All PDF generation agents created successfully!")



✅ All agents updated to use Pexels API!
✅ All PDF generation agents created successfully!


In [22]:


# ============================================
# BLOGGER CONFIGURATION
# ============================================

BLOGGER_BLOG_ID = "4423180991844681752"  # Your blog ID
BLOGGER_CREDENTIALS_FILE = "blogger_credentials1.json"  # Your credentials file
SCOPES = ['https://www.googleapis.com/auth/blogger']

In [23]:
def authenticate_blogger():
    """
    Authenticate with Blogger API using OAuth2 with token caching
    
    Returns:
        Authenticated Blogger API service object
    """
    creds = None
    
    # Check if we have saved credentials
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # If credentials are invalid or don't exist, get new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                BLOGGER_CREDENTIALS_FILE, SCOPES)
            # ✅ Use a FIXED port that matches your console settings
            creds = flow.run_local_server(port=8080)  # Changed from port=0
        
        # Save credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    # Build the Blogger API service
    service = build('blogger', 'v3', credentials=creds)
    return service

In [24]:
def get_blogger_service():
    """Wrapper function for authenticate_blogger"""
    return authenticate_blogger()

print("✅ get_blogger_service() function added!")

✅ get_blogger_service() function added!


In [25]:
def convert_structured_content_to_html(formatted_content: str) -> dict:
    """
    Convert structured content from Blog_PDF_formatter to HTML for Blogger
    
    Args:
        
        formatted_content: Structured content from Blog_PDF_formatter agent
        
    Returns:
        dict with title and html_content
    """
    try:
        lines = formatted_content.strip().split('\n')
        title = ""
        html_content = ""
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            if line.startswith('TITLE:'):
                title = line.replace('TITLE:', '').strip()
                
            elif line.startswith('SECTION:'):
                section_heading = line.replace('SECTION:', '').strip()
                html_content += f"<h2>{section_heading}</h2>\n"
                
            elif line.startswith('CONTENT:'):
                content = line.replace('CONTENT:', '').strip()
                # Split by || to get individual paragraphs
                paragraphs = content.split('||')
                for para in paragraphs:
                    para = para.strip()
                    if para:
                        html_content += f"<p>{para}</p>\n"
                        
            elif line.startswith('IMAGE:'):
                # Extract image description
                image_desc = line.replace('IMAGE:', '').strip()
                # Add as a styled caption/note
                html_content += f"<p style='font-style: italic; color: #666; text-align: center;'>📸 {image_desc}</p>\n"
                
            elif line.startswith('CONCLUSION:'):
                conclusion_text = line.replace('CONCLUSION:', '').strip()
                if conclusion_text:
                    html_content += f"<h2>{conclusion_text}</h2>\n"
                else:
                    html_content += "<h2>Conclusion</h2>\n"
        
        return {
            "success": True,
            "title": title,
            "html_content": html_content
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

In [26]:
def compress_image_for_blogger(image_data, max_size_kb=150, quality=85):
    """
    Compress image to reduce base64 size for Blogger
    
    Args:
        image_data: Raw image bytes
        max_size_kb: Target max size in KB (default 150KB)
        quality: JPEG quality 1-100 (default 85)
    
    Returns:
        Compressed image bytes
    """
    try:
        # Open image
        img = Image.open(BytesIO(image_data))
        
        # Convert RGBA to RGB if needed
        if img.mode == 'RGBA':
            background = Image.new('RGB', img.size, (255, 255, 255))
            background.paste(img, mask=img.split()[3])
            img = background
        elif img.mode != 'RGB':
            img = img.convert('RGB')
        
        # Resize if too large (max 800px width for blog posts)
        max_width = 800
        if img.width > max_width:
            ratio = max_width / img.width
            new_height = int(img.height * ratio)
            img = img.resize((max_width, new_height), Image.Resampling.LANCZOS)
        
        # Compress to target size
        output = BytesIO()
        current_quality = quality
        
        while current_quality > 20:
            output.seek(0)
            output.truncate()
            img.save(output, format='JPEG', quality=current_quality, optimize=True)
            size_kb = len(output.getvalue()) / 1024
            
            if size_kb <= max_size_kb:
                break
            
            current_quality -= 5
        
        print(f"   📉 Compressed: {len(image_data)/1024:.1f}KB → {len(output.getvalue())/1024:.1f}KB")
        return output.getvalue()
        
    except Exception as e:
        print(f"   ⚠️  Compression failed, using original: {e}")
        return image_data


def blogger_post_tool_function_with_dalle_base64(
    formatted_content: str,
    is_draft: bool = True,
    use_dalle: bool = True,
    max_images: int = 3  # Limit images to avoid size issues
) -> str:
    """
    Posts content with optimized DALL-E images using compressed base64
    
    Args:
        formatted_content: Structured content with TITLE, SECTION, CONTENT, IMAGE markers
        is_draft: Whether to post as draft (default: True)
        use_dalle: Whether to generate DALL-E images (default: True)
        max_images: Maximum number of images to embed (default: 3)
    
    Returns:
        str with success/error message and post URL
    """
    try:
        print("\n" + "="*60)
        print("🎨 BLOGGER POST WITH OPTIMIZED DALL-E IMAGES")
        print("="*60)
        print(f"\n📊 Processing content ({len(formatted_content)} chars)...")
        
        lines = formatted_content.strip().split('\n')
        title = ""
        
        # Simple, clean CSS
        html_content = """
        <style>
            .blog-image {
                max-width: 100%;
                height: auto;
                margin: 20px auto;
                display: block;
                border-radius: 8px;
            }
            .image-caption {
                text-align: center;
                font-style: italic;
                color: #666;
                margin: 10px 0 20px 0;
                font-size: 14px;
            }
            h2 {
                color: #2c3e50;
                margin-top: 30px;
                margin-bottom: 15px;
            }
            p {
                line-height: 1.6;
                margin: 10px 0;
            }
        </style>
        """
        
        image_count = 0
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            if line.startswith('TITLE:'):
                title = line.replace('TITLE:', '').strip()
                print(f"📝 Title: {title}")
                
            elif line.startswith('SECTION:'):
                section_title = line.replace('SECTION:', '').strip()
                html_content += f'<h2>{section_title}</h2>'
                
            elif line.startswith('CONTENT:'):
                content = line.replace('CONTENT:', '').strip()
                
                for para in content.split('||'):
                    para = para.strip()
                    if para:
                        # Escape HTML special characters
                        para = para.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                        para = para.replace('&amp;', '&').replace('&lt;', '<').replace('&gt;', '>')
                        html_content += f'<p>{para}</p>'
                        
            elif line.startswith('IMAGE:'):
                # Check if we've reached max images
                if image_count >= max_images:
                    print(f"\n⚠️  Skipping image (max {max_images} reached)")
                    continue
                    
                if use_dalle:
                    # Extract image description
                    image_desc = line.replace('IMAGE:', '').strip()
                    image_desc = re.sub(r'\s*-\s*Source:\s*DALL-E\s*', '', image_desc)
                    
                    # Keep description short
                    if len(image_desc) > 80:
                        image_desc = image_desc[:80] + "..."
                    
                    print(f"\n🎨 Generating image {image_count + 1}: {image_desc[:50]}...")
                    
                    # Generate DALL-E image with SMALLER size
                    try:
                        result = fetch_dalle_image_tool(
                            image_desc, 
                            width=1024,  # Use 1024x1024 instead of larger
                            height=1024,
                            style="natural"  # Use "natural" instead of "vivid" for smaller files
                        )
                        
                        if result.get('success') and result.get('url'):
                            image_url = result['url']
                            print(f"   ⬇️  Downloading image...")
                            
                            # Download the image
                            img_response = requests.get(image_url, timeout=30)
                            img_response.raise_for_status()
                            
                            print(f"   🗜️  Compressing image...")
                            # Compress the image
                            compressed_data = compress_image_for_blogger(
                                img_response.content,
                                max_size_kb=150,  # Target 150KB per image
                                quality=85
                            )
                            
                            # Convert to base64
                            image_base64 = base64.b64encode(compressed_data).decode('utf-8')
                            
                            print(f"   ✅ Image ready ({len(image_base64)/1024:.1f}KB as base64)")
                            
                            # Embed as base64 - PERMANENT!
                            html_content += f'<img src="data:image/jpeg;base64,{image_base64}" class="blog-image" alt="{image_desc}">'
                            html_content += f'<p class="image-caption">✨ {image_desc}</p>'
                            
                            image_count += 1
                            print(f"   ✅ Image {image_count} permanently embedded!")
                            
                            # Rate limiting
                            if image_count < max_images:
                                print("   ⏳ Waiting 3 seconds...")
                                time.sleep(3)
                                
                        else:
                            print(f"   ⚠️  Image generation failed: {result.get('error')}")
                            html_content += f'<p class="image-caption">📸 {image_desc}</p>'
                            
                    except Exception as img_error:
                        print(f"   ⚠️  Image processing failed: {img_error}")
                        html_content += f'<p class="image-caption">📸 {image_desc}</p>'
                else:
                    # Text-only mode
                    image_desc = line.replace('IMAGE:', '').strip()
                    html_content += f'<p class="image-caption">📸 {image_desc}</p>'
                    
            elif line.startswith('CONCLUSION:'):
                conclusion_title = line.replace('CONCLUSION:', '').strip()
                html_content += f'<h2>{conclusion_title if conclusion_title else "Conclusion"}</h2>'
        
        if not title:
            print("❌ ERROR: No title found in content!")
            return "❌ Failed: No title found in formatted content"
        
        print(f"\n📊 Summary:")
        print(f"   - Title: {title}")
        print(f"   - Images embedded: {image_count}")
        print(f"   - Total HTML size: {len(html_content)/1024:.1f}KB")
        
        # Check if HTML is too large
        if len(html_content) > 800000:  # 800KB limit
            print("\n⚠️  WARNING: HTML is very large, may cause issues")
        
        # Post to Blogger
        print(f"\n📤 Posting to Blogger...")
        
        service = get_blogger_service()
        
        # Clean the HTML content
        html_content = html_content.strip()
        
        # Create post body
        post = {
            'kind': 'blogger#post',
            'blog': {'id': BLOGGER_BLOG_ID},
            'title': title,
            'content': html_content
        }
        
        print(f"📋 Post structure:")
        print(f"   - Title: {len(title)} chars")
        print(f"   - Content: {len(html_content)} chars ({len(html_content)/1024:.1f}KB)")
        print(f"   - Images: {image_count}")
        
        # Insert the post
        try:
            request = service.posts().insert(
                blogId=BLOGGER_BLOG_ID,
                body=post,
                isDraft=is_draft
            )
            response = request.execute()
            
            post_url = response.get('url', 'URL not available')
            post_id = response.get('id', 'ID not available')
            
            print(f"\n✅ Blog post {'drafted' if is_draft else 'published'} successfully!")
            print(f"📬 Post URL: {post_url}")
            print(f"🆔 Post ID: {post_id}")
            
            return f"✅ Success! Post URL: {post_url}\nImages embedded: {image_count}"
            
        except HttpError as e:
            error_details = str(e)
            print(f"\n❌ Blogger API error: {error_details}")
            
            # Provide helpful error messages
            if "400" in error_details:
                print("\n💡 Possible causes:")
                print("   - Content too large (try reducing images)")
                print("   - Invalid HTML characters")
                print("   - Blogger API limits exceeded")
            
            return f"❌ Failed: Blogger API error - {error_details}"
            
    except Exception as e:
        error_msg = f"Unexpected error: {str(e)}"
        print(f"\n❌ {error_msg}")
        import traceback
        traceback.print_exc()
        return f"❌ Failed: {error_msg}"


print("✅ Optimized blogger_post_tool_function_with_dalle_base64 created!")

✅ Optimized blogger_post_tool_function_with_dalle_base64 created!


In [27]:
blogger_post_tool = FunctionTool(blogger_post_tool_function_with_dalle_base64)

In [28]:
Blogger_Publisher_Agent = Agent(
    name="Blogger_Publisher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Posts blog content to Blogger with optimized DALL-E images",
    instruction="""Post the formatted blog content to Blogger.
    
    The function will:
    1. Generate up to 3 DALL-E images (to stay within size limits)
    2. Compress images to ~150KB each
    3. Embed as base64 for permanence
    4. Post to Blogger as draft
    
    Call blogger_post_tool_function_with_dalle_base64 with the formatted_content.""",
    tools=[blogger_post_tool],
    output_key="blogger_result"
)

In [29]:
# Create a sequential agent for the full workflow
pdf_Blogger_workflow = SequentialAgent(
    name="pdf_Blogger_workflow",
    sub_agents=[
        root_pdf,  # Your existing blog generation agents
        #PDF_Generator_Agent,
        Blogger_Publisher_Agent
    ]
)

# Run the workflow
runner = InMemoryRunner(agent=pdf_Blogger_workflow)

response = await runner.run_debug("LangKawi tourist destinations")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/prerana/Downloads/Kaggle AI Agents/My_blogging_agent/.venv/lib/python3.11/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > LangKawi tourist destinations
Blog_content > **Langkawi: Island Paradise Awaits!**

Langkawi, often called the "Jewel of Kedah," is a Malaysian archipelago renowned for its stunning natural beauty, duty-free shopping, and a plethora of attractions that cater to every type of traveler. Whether you seek adventure, relaxation, or cultural immersion, Langkawi offers an unforgettable escape.

### **Soar Above the Clouds and Explore Natural Wonders**

One of Langkawi's most iconic experiences is the **Langkawi Cable Car (SkyCab)**, which takes visitors on a breathtaking journey up Gunung Machinchang. From this vantage point, you'll be treated to panoramic views of the lush rainforest and the Andaman Sea. At the summit, you can also walk across the **SkyBridge**, a curved, suspended bridge offering spectacular vistas of the surrounding islands. For those who love to explore nature's artistry, **Seven Wells Waterfall (Telaga Tujuh Waterfall)*


🎨 BLOGGER POST WITH OPTIMIZED DALL-E IMAGES

📊 Processing content (8919 chars)...
📝 Title: Langkawi: Your Ultimate Guide to the Jewel of Kedah

🎨 Generating image 1: Panoramic view from the Langkawi Cable Car, showin...
🎨 Generating DALL-E 3 image...
📝 Description: Panoramic view from the Langkawi Cable Car, showing lush rainforest and the Anda...
⬇️  Downloading high-quality image...
✅ High-quality DALL-E 3 image generated!
   ⬇️  Downloading image...
   🗜️  Compressing image...
   📉 Compressed: 1759.7KB → 132.7KB
   ✅ Image ready (177.0KB as base64)
   ✅ Image 1 permanently embedded!
   ⏳ Waiting 3 seconds...

🎨 Generating image 2: A tranquil freshwater lake surrounded by lush gree...
🎨 Generating DALL-E 3 image...
📝 Description: A tranquil freshwater lake surrounded by lush green hills on an island, with cry...
⬇️  Downloading high-quality image...
✅ High-quality DALL-E 3 image generated!
   ⬇️  Downloading image...
   🗜️  Compressing image...
   📉 Compressed: 2051.3KB → 130.8KB
  